Utilizando o Python para criar um script de carregamento para a "Qualidade"

In [1]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('fato_qualidade.csv')
df.head(2)

,id_controle,id_planta,data,unidades_produzidas,unidades_defeituosas,indice_qualidade
0,1,1,2024-01-01,10229,526,94.86
1,2,2,2024-01-01,12412,724,94.17


Visualizando os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_controle           60 non-null     int64  
 1   id_planta             60 non-null     int64  
 2   data                  60 non-null     object 
 3   unidades_produzidas   60 non-null     int64  
 4   unidades_defeituosas  60 non-null     int64  
 5   indice_qualidade      60 non-null     float64
dtypes: float64(1), int64(4), object(1)
memory usage: 2.9+ KB


Primeiro modo, inserção manual, apenas cria um script de inserção.

In [4]:
with open('inserts_qualidade.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        
        insert_sql = (
            "INSERT INTO produtos (id_controle, id_planta, data, unidades_produzidas, unidades_defeituosas, indice_qualidade) "
            "VALUES ({}, {}, {}, {}, {}, {});\n".format(row['id_controle'], row['id_planta'], row['data'], row['unidades_produzidas'], row['unidades_defeituosas'], row['indice_qualidade'])
        )
        f.write(insert_sql)

print("Arquivo 'inserts_qualidade.sql' gerado com sucesso!")


Arquivo 'inserts_qualidade.sql' gerado com sucesso!


Segundo modo, inserção direta no banco de dados, executa os comandos de insert diretamente.

In [ ]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.

import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'qualidade' e a conexão com o banco de dados.
def insert_data_to_mysql(df, table_name='qualidade'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    data_to_insert = []  # declarar fora do try para retornar depois

    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        # Usando f-string para formatar a consulta SQL com o nome da tabela
        sql = f"INSERT INTO {table_name} (id_controle, id_planta, data, unidades_produzidas, unidades_defeituosas, indice_qualidade) VALUES (%s, %s, %s, %s, %s, %s)"

        # Iterar sobre o DataFrame e preparar os dados para inserção
        # Usando executemany para inserir múltiplos registros de uma vez
        for _, row in df.iterrows():
            values = (
                row['id_controle'],
                row['id_planta'],
                row['data'],
                row['unidades_produzidas'],
                row['unidades_defeituosas'],
                row['indice_qualidade']
            )
            data_to_insert.append(values)

        cursor.executemany(sql, data_to_insert)
        conn.commit()

        #mensagem de sucesso
        print(f"{cursor.rowcount} registros inseridos com sucesso na tabela {table_name}.")
    
    except mysql.connector.Error as err:
        # mensagem de erro
        print(f"Erro MySQL: {err}")
    
    # fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return data_to_insert  # retornando aqui

# Executar e capturar os dados
data_to_insert = insert_data_to_mysql(df)


910 registros inseridos com sucesso na tabela qualidade.
